In [1]:
import pandas as pd
import statsmodels.api as sm # 회귀모델
import joblib # joblib로 모델 저장/모델 load

In [3]:
# 자동완성 핫키 : ctrl+space
df = pd.read_csv('data/apt_api.csv', comment='#', encoding='cp949')
df.sample()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
47,10500,2013,2021,숭인동,종로아인스빌,7,5,12.15,1392-1,11110,15,-,-


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   거래금액     318 non-null    int64  
 1   건축년도     318 non-null    int64  
 2   년        318 non-null    int64  
 3   법정동      318 non-null    object 
 4   아파트      318 non-null    object 
 5   월        318 non-null    int64  
 6   일        318 non-null    int64  
 7   전용면적     318 non-null    float64
 8   지번       318 non-null    object 
 9   지역코드     318 non-null    int64  
 10  층        318 non-null    int64  
 11  해제사유발생일  318 non-null    object 
 12  해제여부     318 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 32.4+ KB


In [12]:
# pd.options.mode.copy_on_write = True
X = df[['건축년도', '전용면적','층']].copy()
X['const'] = 1
y = df['거래금액']
X.shape, y.shape

((318, 4), (318,))

In [ ]:
# 회귀모델
model = sm.OLS(y, X).fit()
model.summary()
# R-squared(r제곱:설명), Adj. R-squared(수정된 r제곱)
# Prob (F-statistic):	8.54e-71 (p-value)
# 독립변수별 coef
# Durbin-Watson : 1.352(자기 상관관계 수치) 2가 이상치

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   거래금액   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     192.4
Date:                Mon, 22 Dec 2025   Prob (F-statistic):           8.54e-71
Time:                        11:07:26   Log-Likelihood:                -3777.5
No. Observations:                 318   AIC:                             7563.
Df Residuals:                     314   BIC:                             7578.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
건축년도        1925.6916    212.616      9.057      0.000    1507.360    2344.023
전용면적         962.1507     47.367     20.313      0.000     868.955    1055.347
층           2058.1524    417.716      4.927      0.000    1236.276    2880.028
const      -3.855e+06   4.25e+05     -9.069      0.000   -4.69e+06   -3.02e+06
==============================================================================
Omnibus:                       20.985   Durbin-Watson:                   1.352
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.734
Skew:                           0.345   Prob(JB):                     5.25e-10
Kurtosis:                       4.658   Cond. No.                     4.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
# 모델 예측
X.iloc[0]
input_data = [[2002, 84, 1, 1]]
round(model.predict(input_data)[0] * 10000)

830001812

In [ ]:
format(round(model.predict(input_data)[0] * 10000), ',') # predict값

'830,001,812'

In [24]:
# 모델 저장
joblib.dump(model, 'model/apt.joblib')

['model/apt.joblib']

In [25]:
import joblib
def predict_apt_price(year, square, floor):
  loaded_model = joblib.load('model/apt.joblib')
  input_data = [[int(year), int(square), int(floor), 1]]
  result = round(model.predict(input_data)[0] * 10000)
  return format(result, ',') + '원입니다'

In [27]:
year = input('건축년도')
square = input('몇 제곱미터?')
floor = input('몇층')
predict_apt_price(year, square, floor)

'1,532,370,126원입니다'